In [1]:
import pickle
import os
import sys
import datetime
import pandas as pd
import numpy as np
import nltk.data
from gensim.models import Word2Vec
import multiprocessing
from copy import deepcopy
from sklearn.metrics import roc_auc_score    
from sklearn.cross_validation import StratifiedKFold
import array
import random

/nlp-vm/anaconda2/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
np.random.seed(10)

In [3]:
import os.path
sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
os.environ['PYTHONHASHSEED']='10'
#is this working?? no idea

In [4]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

### Read in datafile

In [5]:
# Read data from files
df = pd.read_csv( 'labeledTrainData.tsv', header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0,  delimiter="\t", quoting=3 )

In [6]:
df.columns = ['id', 'boolean_label', 'abstract']

### Word2Vec Algorithm- only building basemodel vocab and training on 200 reviews


In [9]:
a = datetime.datetime.now().replace(microsecond=0)

#get indices, 200 for training and 100 for testing randomly subsetted
indices = random.sample(xrange(25000),300)
indices = np.asarray(indices)
print len(indices)
print indices[0:25]
print type(indices)
#split indices into two sets
train_index = indices[0:200]
test_index = indices[200:300]

#we'll build the basemodel on the entire vocabulary, but only train on about 200 reviews

#use the indexes to subset the df pandas dataframe and get the associated rows
train1, test1 = df.iloc[train_index], df.iloc[test_index]
print train1.shape
print test1.shape

sentences = []  # Initialize an empty list of sentences
for j in range(len(train1)):
    review = train1['abstract'].iloc[j]
    sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

print "Building and training w2v models"
## create a w2v learner 
basemodel = Word2Vec(
    sentences=None,
    size=50,
    window=5,
    workers=1, # don't use cores- use single worker thread for fully deterministically-reproducible run
#             workers=multiprocessing.cpu_count(), # use your cores
    iter=1, # iter = sweeps of SGD through the data; more is better
    hs=1, negative=0,
    alpha=0.10# we only have scoring for the hierarchical softmax setup
    #left min_alpha as preset
    )
basemodel.build_vocab(sentences) 

# ****** Split the training set into clean sentences
#
sentences_pos = []  # Initialize an empty list of sentences
sentences_neg = []  # Initialize an empty list of sentences

#here change to include all journal name labels of positive and negative
inxs_pos = np.where(train1['boolean_label'] == 1)[0].tolist()
inxs_neg = np.where(train1['boolean_label'] == 0)[0].tolist()

for inx in inxs_pos:
    review = train1["abstract"].iloc[inx]
    sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

for inx in inxs_neg:
    review = train1["abstract"].iloc[inx]
    sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

#train models
models = [deepcopy(basemodel) for y in range(2)]
models[0].train(sentences_neg, total_examples=len(sentences_neg) )
models[1].train(sentences_pos, total_examples=len(sentences_pos) )

b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

300
[22190 23112 18321 23640  5201  3919  2683  6224  3697  3396  7842 23548
  8923  2552 18627  7834 10064  5625 10267 19770 18399 11077  3519 16556
  6482]
<type 'numpy.ndarray'>
(200, 3)
(100, 3)


/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /nlp-vm/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: "." looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


Parsing sentences from unlabeled set


/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to g

Building and training w2v models
0:04:19


### see how fixed weights change on a single review

In [10]:
print "Parsing test sentences" #into positive and negative groups so know true label
# read in the test set
#NEW: now we are actually using the training set
#and we only need a handful of reviews- 10 of each sent

sentences_pos_test = []  # Initialize an empty list of sentences
sentences_neg_test = []  # Initialize an empty list of sentences
    
inxs_pos_test = np.where(test1['boolean_label'] == 1)[0].tolist()
inxs_neg_test = np.where(test1['boolean_label'] == 0)[0].tolist()


for inx in inxs_pos_test[0:10]:
    review = test1["abstract"].iloc[inx]
    sentences_pos_test += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
    #print inx

for inx in inxs_neg_test[0:10]:
    review = test1["abstract"].iloc[inx]
    sentences_neg_test += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
    #print inx

Parsing test sentences


In [11]:
#save matrices in new variables to load each time, as well as the negative and positive trained models
neg1 = models[0].syn0
neg2 = models[0].syn1
modNeg = models[0]
modPos = models[1]
pos1 = models[1].syn0
pos2 = models[1].syn1

In [12]:
print sentences_pos_test[0]

[u'i', u'am', u'amazed', u'that', u'movies', u'like', u'this', u'can', u'still', u'be', u'made']


In [13]:
#retrain model using a test sentence- 
modPos.train(sentences_pos_test[0], total_examples=1 )
modNeg.train(sentences_pos_test[0], total_examples=1 )

31

### Now look at weight matrices again....how much have they changed?

In [17]:
#look at dims
print neg1.shape
print neg2.shape
print pos1.shape
print pos2.shape

(39795, 50)
(39795, 50)
(39795, 50)
(39795, 50)


### TODO: test that the original weights are actually being stored and not also updated. Then, try for every word in the sentence by recoding the below script.

In [19]:
#all = []
#for sent in sentences_pos_test:
#make copies of models
#    copymodPos = modPos
#    copymodNeg = modNeg
a=[]
#retrain model using a test sentence
#    copymodPos.train(sent, total_examples=1 )
#    copymodNeg.train(sent, total_examples=1 )
#for each word....replace the 0's 

#NOTE: Changed FROM COPYMODNEG TO MODNEG AND SAME FOR MODPOS
for i in range(models[0].syn0.shape[0]):
    a.append(np.subtract(neg1[i], modNeg.syn0[i]))
    a.append(np.subtract(neg2[i], modNeg.syn1[i]))
    a.append(np.subtract(pos1[i], modPos.syn0[i]))
    a.append(np.subtract(pos2[i], modPos.syn1[i]))

    if np.any(np.not_equal(neg1[i],modNeg.syn0[i])):
        print "1"
        print i
        print neg1[i]
        print sent
    elif np.any(np.not_equal(neg2[i],modNeg.syn1[i])):
        print "2"
        print i
        print neg2[i]
        print sent
    elif np.any(np.not_equal(pos1[i],modPos.syn0[i])):
        print "3"
        print i
        print pos1[i]
        print sent
    elif np.any(np.not_equal(pos2[i],modPos.syn1[i])):
        print "4"
        print i
        print pos2[i]
        print sent

#then also append the score to the list*** TODO

#all.append(a)


In [ ]:
#repeat for sentences in sentences_neg_test

In [14]:
print modPos["this"]
print models[1]["this"]
#still identical

[  6.72277287e-02   1.26543835e-01   9.93497670e-03  -5.40592745e-02
  -2.65558302e-01  -9.68429521e-02   8.76064673e-02  -1.33147120e-01
   4.74467762e-02   1.02694966e-01   2.79902313e-02  -5.03830649e-02
  -1.29035428e-01   8.29034764e-03   4.33472246e-02   2.05678880e-01
   1.59809381e-01   3.38250399e-02  -2.10265428e-01  -1.30878299e-01
  -1.95322976e-01  -2.52045747e-02   1.68052465e-01  -4.24853526e-02
   1.19723156e-01   4.54059616e-02   4.93345559e-02  -6.80889115e-02
  -2.11434722e-01  -1.75505623e-01  -1.63846120e-01   8.19133222e-02
   8.09772909e-02   6.10496514e-02   3.02647091e-02   5.76601438e-02
   6.89781224e-03   2.18035445e-01   1.95809379e-01  -1.67943567e-01
   5.93783334e-05   7.96362162e-02  -3.42507352e-04  -8.06212574e-02
   1.97106436e-01  -4.97829355e-02  -9.23664309e-03   1.54408872e-01
  -9.93900597e-02  -1.28648549e-01]
[  6.72277287e-02   1.26543835e-01   9.93497670e-03  -5.40592745e-02
  -2.65558302e-01  -9.68429521e-02   8.76064673e-02  -1.33147120e-0

In [45]:
print modNeg["this"]
print models[0]["this"]
#still identical

[  1.12128623e-01  -5.90351527e-04   1.90263346e-01  -2.04363316e-01
  -1.25054032e-01   1.41420200e-01   2.07533948e-02  -5.16388237e-01
  -2.71282345e-01  -2.74658352e-01   4.18581665e-01   6.85895383e-01
   1.61994889e-01   1.41382843e-01   2.19165564e-01   6.37463620e-03
   1.48713633e-01   4.50969517e-01   2.83410162e-01  -2.03582317e-01
  -1.41550124e-01  -3.66136700e-01   1.05822659e+00  -7.81226233e-02
   3.40196371e-01  -4.13311690e-01  -1.34569913e-01  -5.01110256e-02
   5.74484766e-01   4.05226871e-02   1.15871266e-01  -9.73553479e-01
  -1.10163584e-01  -3.38168532e-01   9.53039765e-01  -1.01441734e-01
   4.13049966e-01   4.90189455e-02  -4.38492864e-01  -5.91962934e-02
  -9.26226377e-01   4.83608961e-01  -7.15108156e-01   3.62209111e-01
   6.68655038e-01   2.67568588e-01   3.60578388e-01   7.26784527e-01
  -2.97102273e-01   4.04141605e-01  -4.31873351e-01  -1.42890409e-01
   3.50398183e-01   3.67146939e-01   7.04841539e-02  -1.48633216e-02
  -1.14153242e+00  -3.92584890e-01

In [8]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)
    
    @staticmethod
    def review_to_words( review, remove_stopwords=False ):
        # Function to convert a raw review to a string of words
        # The input is a single string (a raw movie review), and 
        # the output is a single string (a preprocessed movie review)
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text() 
        #
        # 2. Remove non-letters        
        review_text = re.sub("[^a-zA-Z]", " ", review_text) 
        #
        # 3. Convert to lower case, split into individual words
        words = review_text.lower().split()                             
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]   
        #
        # 6. Join the words back into one string separated by space, 
        # and return the result.
        return( " ".join( words ))   

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences